# Backdoor Attacks-Resilient Aggregation based on Robust Filtering of Outliers in Federated Learning for image classification

In this notebook we provide the implemetation of the averaging mechanism RFOut, presented in [this paper](). We also provide a simple example of how to use it in a federated environment. We use [EMNIST Digits](https://www.nist.gov/itl/products-and-services/emnist-dataset) dataset, a popular dataset for the experiments. We federate the data following an I.I.D. distribution over 20 nodes (clients).

### Data reading

In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import shfl

database = shfl.data_base.Emnist()
train_data, train_labels, test_data, test_labels = database.load_data()

In [2]:
iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_labels = iid_distribution.get_federated_data(num_nodes=20, percent=100)

We reshape training and test data in order to fit the required shape. For reshaping the federated data, we use the class FederatedTransformation. 

In [3]:
import numpy as np

class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        labeled_data.data = np.reshape(labeled_data.data, (labeled_data.data.shape[0], labeled_data.data.shape[1], labeled_data.data.shape[2],1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], test_data.shape[2],1))


### Deep Learning model

We use a simple deep learning model based on two layers of CNN implemented in Keras.

In [4]:
import tensorflow as tf

def model_builder():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1, input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return shfl.model.DeepLearningModel(model)

### Federated Aggregator: RFOut

In this point, we provide the implementation of the RFOut aggregation mechanism. For that purpose, we overwrite the implementation of the FedAvg aggregator in [rfout.py](./rfout.py).


In [5]:
from rfout import RFOut1d

aggregator = RFOut1d()
federated_government = shfl.federated_government.FederatedGovernment(model_builder, federated_data, aggregator)

### Run the federated algorithm

Finally, we run 10 rounds of learning.

In [6]:
federated_government.run_rounds(10, test_data, test_labels)

Accuracy round 0
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f852878b9d0>: [0.2388855218887329, 0.9332000017166138]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f852878b880>: [0.2581176459789276, 0.9220499992370605]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f852878b190>: [0.19621045887470245, 0.9393249750137329]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f85287380a0>: [0.1872367411851883, 0.942550003528595]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f85287382b0>: [0.21427756547927856, 0.9347249865531921]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f84a7bb9250>: [0.22970852255821228, 0.930899977684021]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f84a7bb93a0>: [0.191890731453